### IMPLEMENTATION

In [ ]:
# Connecting spreadsheet to Colab notebook

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
import pandas as pd
import os

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Open spreadsheet and transfer data

#import pandas as pd
#import os

#url = "https://docs.google.com/spreadsheets/d/1r9qTAWrgrN6YX2PFJ7pkOLpAIXfLeVVKOgrxfOQtdlg/edit?usp=sharing"
#df = pd.read_csv(url)

#check if the spreadsheet opened right
#try:
#  sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1r9qTAWrgrN6YX2PFJ7pkOLpAIXfLeVVKOgrxfOQtdlg/edit?usp=sharing")
#  print("Spreadsheet opened properly", sh.title)
#except Exception as e:
#  print("Spreadsheet did not open correctly: {e}", sh.title)

#check if we can get data from our spreadsheet
#try:
#  rows = sh.get_all_values()
#  print(rows)
#except AttributeError as e:
#  print("Data not accessed: {e}")


In [ ]:
worksheet = sh.worksheet('AI Challenge Dataset')
rows = worksheet.get_all_values()

df = pd.DataFrame.from_records(rows)

df.head()

NameError: name 'sh' is not defined

In [ ]:
cols = list(df.columns)
print(cols)

df = df.rename(columns={0: 'brand_id', 1: 'review_id', 2: 'review_dating_by_category', 3: 'ai_aspect', 4: 'ai_sentiment', 5: 'avg_rating', 6: 'is_product_recommended', 7: 'review_content', 8: 'is_anonymous', 9: 'like_count', 10: 'display_name'})
new_cols = list(df.columns)
print(new_cols)

df = df.drop(axis=0, index=0)

df.head()

In [ ]:
df = df.dropna()

df = df.dropna(how='all')
df = df.dropna(axis=1)
df = df.dropna(axis=1, how='all')

df.head()

In [ ]:
import pandas as pd
import json

df_sub = df[['review_dating_by_category']]
df_sub['review_dating_by_category'] = df_sub['review_dating_by_category'].apply(json.loads)
df_exploded = df_sub.explode('review_dating_by_category').reset_index(drop=True)
df_normalized = pd.json_normalize(df_exploded['review_dating_by_category'])
df_final = df_normalized[['id', 'review_id', 'rating_score', 'rating_category']]

df_final.head()

In [ ]:
df_pivot_by_review_id = df_final.pivot_table(index='review_id', columns='rating_category', values='rating_score', aggfunc='first').reset_index()
df_pivot_by_review_id.fillna(0, inplace=True)
df_pivot_by_review_id.head()

In [ ]:
# RUN THIS CELL ONCE (TO SAVE TO YOUR DRIVE INDIVIDUALLY)

#saving cleaned data to pickle file
from google.colab import drive
drive.mount('/content/drive')

df.to_pickle('/content/drive/My Drive/df_pivot_by_review_id.pkl')

In [ ]:
cleaned_data = pd.read_pickle('/content/drive/MyDrive/df_pivot_by_review_id.pkl')
cleaned_data.head()

In [ ]:
#Inspecting EDA results
#import pickle
#from google.colab import drive
# Load the pickle file from Google Drive

#with open('/content/drive/MyDrive/df_pivot_by_review_id.pkl', 'rb') as file:
#    loaded_eda_results = pickle.load(file)

# Inspect the loaded data
#print(loaded_eda_results.keys())  # Shows the keys of the dictionary
#print(loaded_eda_results['missing_values'])  # Example: Access missing values


# ITEM-ITEM COLLABORATIVE FILTERING
- using our cleaned data, calling it by pickle, we will use this to find patterns through an algorithm called collaborative filtering

Here's an explanation of what I did:

## 1. Data Prep:
- Extracts specific rating columns (qualities) and normalizes them to a [0, 1] range for clustering and similarity calculations.
-Ensures the clustering process isn't biased by varying scales across columns.

## 2. Using Clustering with K-Means
- Determines the optimal number of clusters by calculating the "inertia" (sum of squared distances to the nearest cluster center) for different values of k.
- Using the elbow method to visually identify the optimal k.
- Assigns each product to one of the 3 clusters (optimal_k=3) based on their qualities.
- finally, i plotted the `inertia` vs number of clusters and used a heatmap visual to show/determine 'elbow point' or most optimal k

## 3. Cluster Analysis
- Computes the mean ratings for each quality `numeric_columns` within each cluster to summarize the clusters.
- this analysis provides a quick way to inpret and see what clusters represent and its patterns

## 4. Dimensionality Reduction
- Reduces the 6-dimensional quality space to 2 dimensions for easier visualization
- Helps visually interpret cluster separability.
- Visualizes clusters in 2D space, showing which items belong to which cluster.

## 5. Cosine Similarity
- Calculates item-to-item similarity for each cluster using cosine similarity.
- Ensures similarity is computed only within clusters for higher relevance.
- i convert the similarity matrix into a DataFrame for better interpretation and usability.

## 6. Some Sample Recommendations:
- Recommends top n items similar to a given item (item) within a specified cluster
- Uses the precomputed similarity matrix for efficiency

## 6a. Cross Cluster Recs

- Recommends similar items across clusters, excluding the current cluster--> flexibility to refine recommendations by weighing similarities across clusters.

## 6b. Reccommendation for all items

- Automates the process of generating recommendations for all items in a cluster.

## 7. Visualizers
- Visualizes item-to-item similarities within each cluster
- This makes it easier to interpret how qualities relate to each other


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score, pairwise_distances


#Utility Functions
####################################################################################################################################

#Function name: normalize_data
#Function Purpose: normalize the data using a minmaxscalar
#In item-to-item collaborative filtering, "normalizing data" means adjusting the rating values of users on different items so that they are on a comparable scale
def normalize_data(group, columns):
    scaler = MinMaxScaler()
    normalized = pd.DataFrame(scaler.fit_transform(group[columns]), columns=columns, index=group.index)
    return normalized

####################################################################################################################################
#Function name: determine optimal clusters
#Function Purpose: determine the number of clusters using approach of the silhouette method to be more programmatic
def determine_optimal_clusters(data, max_clusters=10):
  inertia = []
  K = range(2, max_clusters+1)
  silhouette_scores = []
  for k in K:
      kmeans = KMeans(n_clusters=k, random_state=42)
      kmeans.fit(qualities_normalized)
      inertia.append(kmeans.inertia_)
      silhouette_scores.append(silhouette_score(qualities_normalized, kmeans.labels_))

  #Plot the elbow curve
  plt.figure(figsize=(8, 5))
  plt.plot(K, inertia, 'bx-', label='Inertia')
  plt.xlabel('Number of Clusters (K)')
  plt.ylabel('Inertia')
  plt.title('Elbow Method for Optimal K')

  # Plot the silhouette curve
  plt.figure(figsize=(8, 5))
  plt.plot(K, silhouette_scores, 'go-')
  plt.xlabel('Number of Clusters (K)')
  plt.ylabel('Silhouette Score')
  plt.title('Silhouette Score for Optimal K')
  plt.show()

  # finding optimal_k
  optimal_k = K[silhouette_scores.index(max(silhouette_scores))]
  print(f"Optimal number of clusters: {optimal_k}")
  return optimal_k

####################################################################################################################################
#Function name: perform_clustering
#Function Purpose: Apply KMeans with optimal clusters and returns labels
def perform_clustering(data, n_clusters):
  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  labels = kmeans.fit_predict(data)
  return kmeans, labels

####################################################################################################################################
#Function name:
#Function Purpose: compute cosine similarity for items in a cluster
def compute_cosine_similarity(group, numeric_columns, metric='cosine'):
    #transpose the qualities to treat them as items
    qualities_transposed = group[numeric_columns].T
    # Compute cosine similarity for qualities within the cluster
    if metric == 'cosine':
        similarity_matrix = 1 - pairwise_distances(qualities_transposed, metric='cosine')
    elif metric == 'euclidean':
        similarity_matrix = 1 / (1 + pairwise_distances(qualities_transposed, metric='euclidean'))
    elif metric == 'correlation':
        similarity_matrix = 1 - pairwise_distances(qualities_transposed, metric='correlation')
    else:
        raise ValueError(f"Unsupported metric: {metric}")
    similarity_df = pd.DataFrame(similarity_matrix, index=numeric_columns, columns=numeric_columns)
    return similarity_df

####################################################################################################################################
#Function name: get_recommendations
#Function Purpose: Get top N similar items for a specific item in a cluster
def get_recommendations(cluster, item, cluster_similarity_results, n=3):
    try:
        similarity_df = cluster_similarity_results[cluster]
        if item not in similarity_df.columns:
            print(f"Item '{item}' not found in cluster {cluster}.")
            return pd.Series(dtype='float64')
        similar_items = similarity_df[item].sort_values(ascending=False)[1:n+1]  # Exclude itself
        return similar_items
    except KeyError:
        print(f"Cluster {cluster} not found in similarity results.")
        return pd.Series(dtype='float64')

####################################################################################################################################
#Function name: plot_pca_clusters
#FunctionPurpose: reduces the dimensionality of `qualities_normalized` data (so from 6D to 2D space)
#the PCA components are stored in columns `PCA1` and `PCA2` of `df_pivot_by_review_id`, and theses columns are only used to show
#the clusters in a 2D space
def plot_pca_clusters(df, pca_components, labels_col):
  plt.figure(figsize=(10, 6))
  for cluster in df_pivot_by_review_id['Cluster'].unique():
      cluster_data = df_pivot_by_review_id[df_pivot_by_review_id['Cluster'] == cluster]
      plt.scatter(cluster_data['PCA1'], cluster_data['PCA2'], label=f'Cluster {cluster}')
  plt.xlabel('PCA1')
  plt.ylabel('PCA2')
  plt.title('Clusters of Product Qualities')
  plt.legend()
  plt.show()

####################################################################################################################################
#Function name:
#Function Purpose: Plot a heatmap of similarity for a specific cluster
def plot_heatmap(cluster, cluster_similarity_results):
    if cluster in cluster_similarity_results:
        similarity_df = cluster_similarity_results[cluster]
        plt.figure(figsize=(8, 6))
        sns.heatmap(similarity_df, annot=True, cmap='coolwarm', fmt='.2f')
        plt.title(f"Item Similarity Heatmap for Cluster {cluster}")
        plt.show()
    else:
        print(f"Cluster {cluster} not found in similarity results.")

####################################################################################################################################


# Main work
####################################################################################################################################

# load Data
df_pivot_by_review_id.head()
qualities = df_pivot_by_review_id[['rating_packaging', 'rating_price', 'rating_quality',
                                   'rating_service', 'rating_shipping', 'rating_taste']]

#Checking distribution of overall features
print("CHECKING DISTRIBUTION ............")
print("Overall distribution of numeric columns:")
print(qualities.describe())

# normalize the data
qualities_normalized = normalize_data(qualities,columns=qualities.columns)

#Just trying things out: Let's see what happens if I drop low-variance features
print("DROPPING LOW VARIANCE FEATURES ............")
low_variance_columns = [col for col in qualities.columns if qualities[col].var() < 0.01]
if low_variance_columns:
    print("THE LOW VARIANCE DROPPING: ")
    print(f"Dropping low-variance columns: {low_variance_columns}")
    qualities_normalized = qualities_normalized.drop(columns=low_variance_columns)

# determine optimal number of clusters
optimal_k = determine_optimal_clusters(qualities_normalized)

# perform clustering
kmeans, labels = perform_clustering(qualities_normalized, n_clusters=optimal_k)
df_pivot_by_review_id['Cluster'] = labels

# analyze the clusters
numeric_columns = qualities_normalized.columns
cluster_similarity_results = {}
for cluster, group in df_pivot_by_review_id.groupby('Cluster'):
    if len(group) <= 1 or group[numeric_columns].var().sum() == 0:
        print(f"Cluster {cluster} has too few items or low variance.")
        cluster_similarity_results[cluster] = pd.DataFrame(0, index=numeric_columns, columns=numeric_columns)
    else:
        cluster_similarity_results[cluster] = compute_cosine_similarity(group, numeric_columns)


# PCA for visualization (not cleaning the data/reducing dimensionality)
pca = PCA(n_components=2)
qualities_pca = pca.fit_transform(qualities_normalized)
df_pivot_by_review_id['PCA1'] = qualities_pca[:, 0]
df_pivot_by_review_id['PCA2'] = qualities_pca[:, 1]
plot_pca_clusters(df_pivot_by_review_id, ['PCA1', 'PCA2'], 'Cluster')

# compute cosine similarity within clusters
cluster_similarity_results = {}
for cluster, group in df_pivot_by_review_id.groupby('Cluster'):
    if len(group) <= 1 or group[numeric_columns].var().sum() == 0:
        print(f"Cluster {cluster} has too few items or low variance for similarity calculation.")
        cluster_similarity_results[cluster] = pd.DataFrame(0, index=numeric_columns, columns=numeric_columns)
    else:
        normalized_group = normalize_data(group, numeric_columns)
        cluster_similarity_results[cluster] = compute_cosine_similarity(normalized_group, numeric_columns)

# generate recommendations for all items in each cluster
all_cluster_recommendations = {}
for cluster in cluster_similarity_results.keys():
    print(f"Generating recommendations for Cluster {cluster}...")
    cluster_recommendations = {item: get_recommendations(cluster, item, cluster_similarity_results) for item in numeric_columns}
    all_cluster_recommendations[cluster] = cluster_recommendations

# display all recommendations
print("\nRecommendations for all qualities in Cluster 0:")
for item, recommendations in all_cluster_recommendations[0].items():
    print(f"Top 3 recommendations for '{item}':")
    print(recommendations)
    print()

# Generate heatmaps for all clusters
for cluster in cluster_similarity_results.keys():
    plot_heatmap(cluster, cluster_similarity_results)

# Doing some modeling things type shi -
lol

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
import spacy
from spacy.training import Example
import random
import nltk
import re

# Ensure required NLTK resources are downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords

# Sample DataFrame for reviews
df_reviews = pd.DataFrame({
    'review_content': [
        "The packaging was secure and beautiful, but the price felt too high for the quality.",
        "The taste was amazing, and it arrived on time.",
        "The service was helpful, but the packaging was damaged.",
        "The shipping was fast, and the product was affordable.",
        "The quality of the item exceeded expectations, but the packaging was unattractive."
    ]
})

### 1. AI Sentiment Analysis ###
# Predefined categories with associated keywords
categories = {
    "packaging": ["packaging", "box", "wrap", "secure", "damaged", "design"],
    "quality": ["quality", "durable", "sturdy", "poorly made"],
    "taste": ["taste", "flavor", "delicious", "bland"],
    "service": ["service", "helpful", "rude", "customer support"],
    "shipping": ["shipping", "fast", "delayed", "broken"],
    "price": ["price", "affordable", "expensive", "worth", "value"]
}

# Categorize reviews based on keywords
def categorize_review(review):
    scores = {category: 0 for category in categories}
    for category, keywords in categories.items():
        if any(keyword in review.lower() for keyword in keywords):
            analysis = TextBlob(review)
            scores[category] = analysis.sentiment.polarity
    return scores

# Apply categorization to the reviews
df_reviews['categorized_scores'] = df_reviews['review_content'].apply(categorize_review)
categorized_df = pd.json_normalize(df_reviews['categorized_scores'])
df_reviews = pd.concat([df_reviews, categorized_df], axis=1)

# Visualize AI Sentiment
plt.figure(figsize=(8, 5))
categorized_df.mean().plot(kind='barh', color=sns.color_palette('muted'))
plt.xlabel('Average Sentiment Score')
plt.ylabel('Categories')
plt.title('AI Sentiment Analysis by Categories')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.show()

### 2. TF-IDF Vectorization and KMeans Clustering ###
# Preprocess reviews
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.lower()  # Convert to lowercase
    words = [word for word in text.split() if word not in stop_words]
    return ' '.join(words)

df_reviews['cleaned_reviews'] = df_reviews['review_content'].apply(preprocess_text)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_df=0.85, min_df=2, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_reviews['cleaned_reviews'])

# KMeans Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df_reviews['Cluster'] = kmeans.fit_predict(tfidf_matrix)

print("\nCluster Assignments:")
print(df_reviews[['review_content', 'Cluster']])

### 3. Topic Modeling with LDA ###
# Fit LDA Model
lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(tfidf_matrix)

# Display LDA Topics: Linear discriminant analysis (LDA) is an approach used in supervised machine learning
#to solve multi-class classification problems. LDA separates multiple classes with multiple features through data dimensionality reduction
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

print("\nLDA Topics:")
display_topics(lda, vectorizer.get_feature_names_out(), no_top_words=5)

### 4. Named Entity Recognition (NER) ###
# Load a blank spaCy model and add NER pipeline
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Define training data
TRAIN_DATA = [
    ("The packaging was secure.", {"entities": [(4, 13, "CATEGORY")]}),
    ("The price was too high.", {"entities": [(4, 9, "CATEGORY")]}),
    ("The service was amazing.", {"entities": [(4, 11, "CATEGORY")]}),
    ("The shipping was fast.", {"entities": [(4, 12, "CATEGORY")]}),
    ("The quality was excellent.", {"entities": [(4, 11, "CATEGORY")]}),
]

# Add labels to NER
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Train the NER model
# Named entity recognition (NER) is a natural language processing (NLP) method that extracts information from text.
# NER involves detecting and categorizing important information in text known as named entities.
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*unaffected_pipes):
    optimizer = nlp.begin_training()
    for i in range(10):  # Number of iterations
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = spacy.util.minibatch(TRAIN_DATA, size=2)
        for batch in batches:
            examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in batch]
            nlp.update(examples, drop=0.5, losses=losses)
        print(f"Iteration {i}: Losses {losses}")

# Test NER model
test_text = "The packaging was secure and the service was helpful."
doc = nlp(test_text)
print("\nNER Results:")
print([(ent.text, ent.label_) for ent in doc.ents])


# BACKEND IMPLEMENTATION

## Installing Necessary Libraries

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [ ]:
#Important Libraries and Initialize Fast API
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from enum import Enum
from pyngrok import ngrok
from fastapi.responses import FileResponse
import uvicorn
import nest_asyncio
import difflib
import gspread
from google.oauth2.service_account import Credentials
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


# !!! DO NOT DELETE CODE BELOW!!!!!

# HERE IS THE WEBSITE TO CREATE NGROK TOKEN: https://dashboard.ngrok.com/get-started/your-authtoken

class TeamMember(Enum):
    SOPHIE = 0
    DIVYA = 1
    HALEY = 2
    YINA = 3
#

#  !!!!!! PLEASE SEE MY TEXT MESSAGE ON CREATING A TOKEN IN THE GC !!!!!!
# PASTE YOUR OWN TOKEN BELOW
team_tokens = [
    "2poAIbS9btZy4wEGBn6M7hmKV1s_7HY2EY7D4huxndyyNnWeW",  # Sophies Token-ID :0
    "authtoken_2", #divya token
    "authtoken_3", #haley toke
    "authtoken_4", #yina token
]

df_reviews = pd.DataFrame({
    "review_id": [1, 2, 3, 4, 5],
    "review_content": [
        "The packaging was secure and beautiful, but the price felt too high for the quality.",
        "The taste was amazing, and it arrived on time.",
        "The service was helpful, but the packaging was damaged.",
        "The shipping was fast, and the product was affordable.",
        "The quality of the item exceeded expectations, but the packaging was unattractive."
    ]
})

# Combine Google Sheets data with the initial dataset
try:
    worksheet = sh.worksheet('AI Challenge Dataset')
    rows = worksheet.get_all_values()
    df = pd.DataFrame.from_records(rows)

    df.head()
    #df_google_sheets = pd.DataFrame.from_records(rows)
    # Convert Google Sheets data to DataFrame
    #df_google_sheets = pd.DataFrame(rows[1:], columns=rows[0])  # First row is the header

    # Extract only `review_id` and `review_content`
    df_google_sheets = df[['review_id', 'review_content']].dropna()

    # Ensure `review_id` is numeric
    df_google_sheets['review_id'] = pd.to_numeric(df_google_sheets['review_id'], errors='coerce')
    df_google_sheets = df_google_sheets.dropna(subset=['review_id'])  # Remove rows with invalid review_id
    df_google_sheets['review_id'] = df_google_sheets['review_id'].astype(int)

    # Combine with the original `df_reviews`
    df_reviews = pd.concat([df_reviews, df_google_sheets], ignore_index=True)
    print("Google Sheets data loaded and combined successfully.")
    print("Google Sheets data loaded and combined successfully.")
except Exception as e:
    print(f"Failed to load Google Sheets data: {e}")

# Select the team member
current_member = TeamMember.SOPHIE  # Change this dynamically based on context

# Get the selected token using the Enum value
selected_token = team_tokens[current_member.value]


# Set the ngrok authtoken
ngrok.set_auth_token(selected_token)

# Load a trained NER model or create one
# Load a trained NER model or train one
MODEL_PATH = "ner_model"

try:
    nlp = spacy.load(MODEL_PATH)
    print("Loaded pre-trained NER model.")
except Exception as e:
    print(f"Error loading model: {e}. Training a new model...")
    nlp = spacy.blank("en")
    ner = nlp.add_pipe("ner")

    TRAIN_DATA = [
        ("The packaging was secure and beautiful.", {"entities": [(4, 13, "CATEGORY")]}),
        ("The price felt too high for the quality.", {"entities": [(4, 9, "CATEGORY")]}),
        ("The taste was amazing.", {"entities": [(4, 9, "CATEGORY")]}),
        ("Customer service was very helpful.", {"entities": [(9, 16, "CATEGORY")]}),
        ("Shipping was fast and reliable.", {"entities": [(0, 8, "CATEGORY")]}),
        ("The product quality exceeded expectations.", {"entities": [(12, 19, "CATEGORY")]}),
    ]


    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*unaffected_pipes):
        optimizer = nlp.begin_training()
        for i in range(10):  # Number of iterations
            batches = spacy.util.minibatch(TRAIN_DATA, size=2)
            for batch in batches:
                examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in batch]
                nlp.update(examples, drop=0.5)
        nlp.to_disk(MODEL_PATH)  # Save model for future use
        print("New model trained and saved.")

# Im testing this out but using our recommendation logic (our models - item to item and basic NER model)
# Initialize FastAPI app
app = FastAPI()

@app.get("/favicon.ico")
async def favicon():
    return FileResponse("path/to/favicon.ico")


# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")


# Extract the domain from the public URL
#ngrok_url = public_url.public_url  # e.g., "https://abcd-1234.ngrok-free.app"

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Dynamically set the allowed origin
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# work
# Map frontend categories to backend model columns
category_mapping = {
    'packaging': 'rating_packaging',
    'price': 'rating_price',
    'quality': 'rating_quality',
    'service': 'rating_service',
    'shipping': 'rating_shipping',
    'taste': 'rating_taste'
}


# Data model for incoming requests
class RecommendationRequest(BaseModel):
    category: str

# Utility functions for ML pipeline
def is_similar(a, b, threshold=0.8):
    return difflib.SequenceMatcher(None, a, b).ratio() > threshold

def get_review_content_for_category(recommendations):
    relevant_reviews = []
    for recommendation in recommendations:
        for _, row in df_reviews.iterrows():
            doc = nlp(row["review_content"])
            print(f"Review: {row['review_content']}")
            print(f"Detected Entities: {[ent.text for ent in doc.ents]}")

            for ent in doc.ents:
                if ent.label_ == "CATEGORY" and is_similar(recommendation.lower(), ent.text.lower()):
                    print(f"Matching Entity: {ent.text}")
                    relevant_reviews.append(row["review_content"])
    if not relevant_reviews:
        relevant_reviews.append("No relevant reviews found.")

def normalize_data(group, columns):
    scaler = MinMaxScaler()
    normalized = pd.DataFrame(scaler.fit_transform(group[columns]), columns=columns, index=group.index)
    return normalized

def compute_cosine_similarity(group, numeric_columns, metric='cosine'):
    qualities_transposed = group[numeric_columns].T
    if metric == 'cosine':
        similarity_matrix = 1 - pairwise_distances(qualities_transposed, metric='cosine')
    else:
        raise ValueError(f"Unsupported metric: {metric}")
    similarity_df = pd.DataFrame(similarity_matrix, index=numeric_columns, columns=numeric_columns)
    return similarity_df

# ML pipeline logic
def generate_recommendations_no_pivot(category):
    print("calling generate_recommendation")  # Debugging log

    # Ensure expected columns are present
    expected_columns = ['rating_packaging', 'rating_price', 'rating_quality', 'rating_service', 'rating_shipping', 'rating_taste']
    df_pivot_by_review_id = pd.DataFrame(columns=expected_columns).fillna(0)

    # Add dummy data if the DataFrame is empty
    if df_pivot_by_review_id.empty:
        print("Input DataFrame is empty. Adding default data.")
        df_pivot_by_review_id = pd.DataFrame({
            'rating_packaging': [3, 4, 5],
            'rating_price': [4, 5, 2],
            'rating_quality': [5, 2, 3],
            'rating_service': [3, 5, 4],
            'rating_shipping': [4, 3, 5],
            'rating_taste': [2, 4, 5],
        })

    # Normalize data
    qualities = df_pivot_by_review_id[expected_columns]
    if qualities.empty:
        print("No data to normalize.")
        return {"recommendations": [], "review_content": []}

    qualities_normalized = normalize_data(qualities, columns=qualities.columns)

    # Perform clustering
    kmeans = KMeans(n_clusters=2, random_state=42)
    df_pivot_by_review_id['Cluster'] = kmeans.fit_predict(qualities_normalized)

    # Filter by cluster (example: Cluster 0)
    cluster_data = df_pivot_by_review_id[df_pivot_by_review_id['Cluster'] == 0]

    # Compute cosine similarity
    similarity_matrix = cosine_similarity(qualities_normalized.T)
    similarity_df = pd.DataFrame(similarity_matrix, index=expected_columns, columns=expected_columns)

    # Generate recommendations
    recommendations = similarity_df[category].sort_values(ascending=False).index.tolist()[:3]

    # Fetch relevant review content
    review_content = get_review_content_for_category(recommendations)

    return {"recommendations": recommendations, "review_content": review_content}

def generate_recommendations(category):
    print("calling generate_recommendations")  # Debugging log

    # Pivot DataFrame
    expected_columns = ["rating_packaging", "rating_price", "rating_quality", "rating_service", "rating_shipping", "rating_taste"]
    df_pivot_by_review_id = pd.DataFrame(columns=expected_columns).fillna(0)

    # Add dummy data if the DataFrame is empty
    if df_pivot_by_review_id.empty:
        print("Input DataFrame is empty. Adding default data.")
        df_pivot_by_review_id = pd.DataFrame({
            "review_id": [1, 2, 3, 4, 5],
            "rating_packaging": [5, 0, 4, 2, 3],
            "rating_price": [4, 3, 5, 1, 2],
            "rating_quality": [3, 5, 0, 4, 2],
            "rating_service": [2, 4, 5, 3, 1],
            "rating_shipping": [1, 2, 3, 5, 4],
            "rating_taste": [0, 5, 4, 3, 2],
        })

    # Merge review_content into df_pivot_by_review_id
    df_combined = pd.merge(df_pivot_by_review_id, df_reviews, on="review_id", how="left")

    # Normalize data
    scaler = MinMaxScaler()
    qualities = df_combined[expected_columns]
    qualities_normalized = pd.DataFrame(scaler.fit_transform(qualities), columns=qualities.columns, index=qualities.index)

    # Perform clustering
    kmeans = KMeans(n_clusters=2, random_state=42)
    df_combined["Cluster"] = kmeans.fit_predict(qualities_normalized)

    # Filter by cluster (example: Cluster 0)
    cluster_data = df_combined[df_combined["Cluster"] == 0]

    # Compute cosine similarity
    similarity_matrix = cosine_similarity(qualities_normalized.T)
    similarity_df = pd.DataFrame(similarity_matrix, index=expected_columns, columns=expected_columns)

    # Generate recommendations
    recommendations = similarity_df[category].sort_values(ascending=False).index.tolist()[:3]

    # Fetch relevant reviews
    relevant_reviews = cluster_data.loc[:, ["review_content"]].dropna()["review_content"].tolist()

    return {"recommendations": recommendations, "review_content": relevant_reviews}

# Endpoint to fetch recommendations
@app.post("/recommendations/")
async def get_recommendations(request: RecommendationRequest):
    try:
        category = request.category
        if category not in category_mapping:
            raise ValueError(f"Invalid category: {category}")

        mapped_category = category_mapping[category]
        result = generate_recommendations(mapped_category)

        if not result["recommendations"]:
            return {"status": "error", "message": "No recommendations found"}

        return {
            "status": "success",
            "data": result
        }

    except ValueError as ve:
        print(f"Validation Error: {ve}")
        raise HTTPException(status_code=400, detail=str(ve))
    except Exception as e:
        print(f"Error: {str(e)}")
        raise HTTPException(status_code=500, detail="Internal Server Error")


# work
#allows for multiple event loops
nest_asyncio.apply()


# Start the FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
#from fastapi.responses import FileResponse
#import uvicorn
#import nest_asyncio

#@app.get("/favicon.ico")
#async def favicon():
#    return FileResponse("path/to/favicon.ico")

In [ ]:
#importing my authtoken:
#from pyngrok import ngrok
#from pyngrok import ngrok
#import uvicorn
#import nest_asyncio

# Replace 'YOUR_AUTH_TOKEN' with the authtoken you copied from ngrok
#ngrok.set_auth_token("2poAIbS9btZy4wEGBn6M7hmKV1s_7HY2EY7D4huxndyyNnWeW")

#retry running ngrok
# Allow multiple event loops
#nest_asyncio.apply()

# Expose the FastAPI app
#public_url = ngrok.connect(8000)
#print(f"Public URL: {public_url}")

# Start the FastAPI server
#uvicorn.run(app, host="0.0.0.0", port=8000)


In [ ]:
#@app.post("/recommendations/")
#async def get_recommendations(request: RecommendationRequest):
#    category = request.category
#    print(f"Received category: {category}")  # Log received data
#    recommendations = get_recommendations_for_category(category)
#    if not recommendations:
#        raise HTTPException(status_code=404, detail="Category not found")
#    print(f"Returning recommendations: {recommendations}")  # Log response
#    return {"recommendations": recommendations}

In [ ]:
# Looking further into API endpoint
#import requests

#url = 'https://1a7b-34-16-216-174.ngrok-free.app/recommendations/'
#payload = {'category': 'packaging'}
#headers = {'Content-Type': 'application/json'}

#response = requests.post(url, json=payload, headers=headers)

# Print the response to debug
#print('Status Code:', response.status_code)
#print('Response JSON:', response.json())

In [ ]:
#Authenticate the ngrok agent
#ngrok config add-authtoken 2poAIbS9btZy4wEGBn6M7hmKV1s_7HY2EY7D4huxndyyNnWeW

# Developing Front-End code in VSCODE